# LLM Zoomcamp HW4

## Getting the data

In [16]:
import pandas as pd
import numpy as np

In [17]:
github_url = "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/04-monitoring/data/results-gpt4o-mini.csv"

In [18]:
url = f"{github_url}?raw=1"
df = pd.read_csv(url)

In [19]:
df = df.iloc[:300]

## Q1. Getting the embeddings model

In [20]:
model_name = "multi-qa-mpnet-base-dot-v1"

In [21]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(model_name)

In [22]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [23]:
answer_llm = df.iloc[0].answer_llm

In [24]:
v_llm = embedding_model.encode(answer_llm)
v_llm[0]

-0.42244655

# Q2. Computing the dot product

In [25]:
records = df.to_dict(orient="records")

In [26]:
records[0]

{'answer_llm': 'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).',
 'answer_orig': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'document': '0227b872',
 'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp'}

In [27]:
def compute_similarity(record):
    answer_llm = record["answer_llm"]
    answer_orig = record["answer_orig"]

    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)

    return v_llm.dot(v_orig)

In [28]:
evaluations = []

for record in records:
    dot_product = compute_similarity(record)
    evaluations.append(dot_product)

In [30]:
df["cosine_sim"] = evaluations

In [31]:
df["cosine_sim"].describe()["75%"]

31.67430877685547

## Q3. Computing the cosine

In [32]:
rec_test = records[0]
rec_test

{'answer_llm': 'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).',
 'answer_orig': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'document': '0227b872',
 'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp'}

In [33]:
def compute_norm(vector):
    norm = np.sqrt((vector * vector).sum())
    return vector / norm

In [34]:
def compute_similarity_with_norm(record):
    answer_llm = record["answer_llm"]
    answer_orig = record["answer_orig"]

    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)

    v_llm = compute_norm(v_llm)
    v_orig = compute_norm(v_orig)

    return v_llm.dot(v_orig)

In [35]:
evaluations_norm = []

for record in records:
    dot_product_norm = compute_similarity_with_norm(record)
    evaluations_norm.append(dot_product_norm)

In [36]:
df["cosine_sim_norm"] = evaluations_norm

df["cosine_sim_norm"].describe()["75%"]

0.8362348973751068

## Q4. Rouge

In [37]:
r = df.iloc[10]

In [38]:
from rouge import Rouge

rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r["answer_llm"], r["answer_orig"])[0]

In [39]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

## Q5. Average rouge score

In [40]:
rouge_1 = scores["rouge-1"]["f"]
rouge_2 = scores["rouge-2"]["f"]
rouge_l = scores["rouge-l"]["f"]
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

rouge_avg

0.35490034990035496

## Q6. Average rouge score for all the data points

In [41]:
df

,answer_llm,answer_orig,document,question,course,cosine_sim,cosine_sim_norm
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,17.515987,0.506754
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,13.418402,0.388549
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,25.313255,0.718599
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,12.147415,0.337266
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,18.747736,0.521792
...,...,...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp,34.001778,0.914175
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp,33.690861,0.902190
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp,34.491524,0.904734
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp,27.538357,0.726782


In [42]:
def calculate_rouge(row):
    rouge_scorer = Rouge()

    scores = rouge_scorer.get_scores(row["answer_llm"], row["answer_orig"])[0]

    rouge_1 = scores["rouge-1"]["f"]
    rouge_2 = scores["rouge-2"]["f"]
    rouge_l = scores["rouge-l"]["f"]

    return (rouge_1 + rouge_2 + rouge_l) / 3

In [43]:
df["rouge_score"] = df.apply(calculate_rouge, axis=1)

In [45]:
df["rouge_score"].describe()

count    300.000000
mean       0.313205
std        0.158133
min        0.000000
25%        0.197358
50%        0.298640
75%        0.404169
max        0.813043
Name: rouge_score, dtype: float64